In [1]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
import numpy as np

# generate some sample data
X = np.random.rand(100, 10)
y = np.random.rand(100)

# create a Ridge model and fit to the data
ridge_model = Ridge(alpha=0.5)
ridge_model.fit(X, y)

# calculate the score and coefficients
score = ridge_model.score(X, y)
coefficients = ridge_model.coef_

print("R^2 score:", score)
print("Coefficients:", coefficients)

R^2 score: 0.09131650418520776
Coefficients: [ 0.05876109  0.03658365 -0.11698    -0.11173144  0.0717869   0.12942832
 -0.06841139 -0.06184682 -0.01805924 -0.10640997]


In [2]:
from sklearn.linear_model import RidgeCV
from sklearn.datasets import load_boston

# Load data
boston = load_boston()

# Split data into features and target variable
X, y = boston.data, boston.target

# Initialize RidgeCV with the range of alpha values to test
alphas = [0.1, 1.0, 10.0]
ridge = RidgeCV(alphas=alphas)

# Fit the model using cross-validation
ridge.fit(X, y)

# Print the optimal value of alpha
print("Optimal value of alpha:", ridge.alpha_)


Optimal value of alpha: 0.1


/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Ca

In [3]:
?RidgeCV

Init signature:
RidgeCV(
    alphas=(0.1, 1.0, 10.0),
    *,
    fit_intercept=True,
    normalize='deprecated',
    scoring=None,
    cv=None,
    gcv_mode=None,
    store_cv_values=False,
    alpha_per_target=False,
)
Docstring:     
Ridge regression with built-in cross-validation.

See glossary entry for :term:`cross-validation estimator`.

By default, it performs efficient Leave-One-Out Cross-Validation.

Read more in the :ref:`User Guide <ridge_regression>`.

Parameters
----------
alphas : ndarray of shape (n_alphas,), default=(0.1, 1.0, 10.0)
    Array of alpha values to try.
    Regularization strength; must be a positive float. Regularization
    improves the conditioning of the problem and reduces the variance of
    the estimates. Larger values specify stronger regularization.
    Alpha corresponds to ``1 / (2C)`` in other linear models such as
    :class:`~sklearn.linear_model.LogisticRegression` or
    :class:`~sklearn.svm.LinearSVC`.
    If using Leave-One-Out cross-valida